#### **Tema 7 - Ingeniería de características**

**Representacion del Problema**

Se decidió representar el mapa a colorear a traves de un grafo, donde cada nodo es una provinicia y tiene asigando utilizar una lista de adyacencia:


In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

provincias = [
'Buenos Aires',
'Catamarca',
'Chaco',
'Chubut',
'Córdoba',
'Corrientes',
'Entre Ríos',
'Formosa',
'Jujuy',
'La Pampa',
'La Rioja',
'Mendoza',
'Misiones',
'Neuquén',
'Río Negro',
'Salta',
'San Juan',
'San Luis',
'Santa Cruz',
'Santa Fe',
'Santiago del Estero',
'Tierra del Fuego',
'Tucumán']
adjProvincias = {}
adjProvincias['Jujuy'] = ['Salta']
adjProvincias['Salta'] = ['Jujuy', 'Catamarca', 'Tucumán', 'Santiago del Estero','Chaco', "Formosa"]
adjProvincias['Formosa'] = ['Chaco', "Salta"]
adjProvincias['Chaco'] = ['Salta', "Santiago del Estero", "Santa Fe", "Corrientes", "Formosa"]
adjProvincias['Catamarca'] = ['Tucumán', "La Rioja", "Salta", "Santiago del Estero", "Córdoba"]
adjProvincias['Tucumán'] = ["Salta", "Catamarca", "Santiago del Estero"]
adjProvincias["Santiago del Estero"] = ["Salta", "Tucumán", "Catamarca", "Córdoba", "Chaco", "Santa Fe"]
adjProvincias["Santa Fe"] = ["Chaco", "Corrientes", "Entre Ríos", "Buenos Aires", "Córdoba", "Santiago del Estero"]
adjProvincias['Corrientes'] = ['Misiones', "Entre Ríos", "Santa Fe", "Chaco"]
adjProvincias['Misiones'] = ['Corrientes']
adjProvincias['La Rioja'] = ['Catamarca', 'Córdoba', "San Luis", "San Juan"]
adjProvincias["Córdoba"] = ["Santiago del Estero", "Santa Fe", "Buenos Aires", "La Pampa", "San Luis", "La Rioja", "Catamarca"]
adjProvincias["Entre Ríos"] = ["Santa Fe", "Corrientes", "Buenos Aires"]
adjProvincias["San Juan"] = ["La Rioja", "San Luis", "Mendoza"]
adjProvincias["San Luis"] = ["La Rioja", "Córdoba", "La Pampa", "Mendoza", "San Juan"]
adjProvincias["Mendoza"] = ["San Juan", "San Luis", "La Pampa", "Neuquén", "Río Negro"]
adjProvincias["La Pampa"] = ["San Luis", "Córdoba", "Buenos Aires", "Río Negro", "Neuquén", "Mendoza"]
adjProvincias["Buenos Aires"] = ["Santa Fe", "Entre Ríos", "Río Negro", "La Pampa", "Córdoba"]
adjProvincias["Neuquén"] = ["Mendoza", "La Pampa", "Río Negro"]
adjProvincias["Río Negro"] = ["Neuquén", "Mendoza", "La Pampa", "Buenos Aires", "Chubut"]
adjProvincias["Chubut"] = ["Río Negro", "Santa Cruz"]
adjProvincias["Santa Cruz"] = ["Chubut", "Tierra del Fuego"]
adjProvincias["Tierra del Fuego"] = ["Santa Cruz"]


print("Validadndo lista de adyacencias...")
for i in provincias:
    if i not in adjProvincias:
        raise Exception(f"La provinicia {i} no se encuentra dentro de la lista de adyacencia")
    for j in adjProvincias[i]:
        if i not in adjProvincias[j]:
            raise Exception(f"Las adyacencias no son simetricas entre: {i} - {j}")
print("Validación exitosa!")


Validadndo lista de adyacencias...
Validación exitosa!


In [92]:
def fitness(nodes, edges, colors):
    fitness = 0
    uniqueColors = set()
    for i in range(len(nodes)):
        for j in range(len(nodes)):
            if i==j:
                continue
            if nodes[j] not in edges[nodes[i]]:
                continue
            if colors[i] == colors[j]:
                return -100 #Numero artificialmente bajo para descartar soluciones invalidas
        uniqueColors.add(colors[i])
    return -len(uniqueColors)

def initializePopulation(size, colorQuantity):
    population = np.ndarray((size, 23))
    for i in range(size):
        for j in range(23): 
            population[i][j] = random.randint(1, colorQuantity)
    return population

def selectionTournament(population, fitnesses, tournamentSize):
    selected = []
    for _ in range(len(population)):
        tournamentParticipants = random.sample(list(zip(population, fitnesses)), tournamentSize)
        winner = max(tournamentParticipants, key= lambda x: x[1])
        selected.append(winner[0])
    return selected

def crossover_one_point(parent1, parent2, crossoverProbability):
    crossPoint = random.randint(0, len(parent1) - 2)
    if random.random() < crossoverProbability: 
        child1 = np.append(parent1[:crossPoint+1], parent2[crossPoint + 1:])
        child2 = np.append(parent2[:crossPoint+1], parent1[crossPoint + 1:])
        return child1, child2
    return parent1, parent2

def mutation(individual, mutationRate, colorQuantity): 
    result = individual[:] #Creo una copia
    for i in range(len(individual)):
        if random.random() < mutationRate:
            individual[i] = random.randint(1, colorQuantity)
    return result

In [97]:
populationSize = 50
availableColors = 36
generations = 50
tournamentSize = 4
mutationRate = 0.01
crossoverProbability = 0.9

population = initializePopulation(populationSize, availableColors)

allPopulations = []
bestPerformers = []

for generation in range(generations):
    fitnesses = [fitness(provincias, adjProvincias, ind) for ind in population]

    best_individual = max(population, key=lambda x : fitness(provincias, adjProvincias, x))
    best_fitness = fitness(provincias, adjProvincias, best_individual)
    bestPerformers.append((best_individual, best_fitness))
    allPopulations.append(population[:])

    selectedParents = selectionTournament(population, fitnesses, tournamentSize)

    next_population = []
    for i in range(0, len(population), 2):
        parent1 = selectedParents[i]
        parent2 = selectedParents[i+1]

        child1, child2 = crossover_one_point(parent1, parent2, crossoverProbability)

        next_population.append(mutation(child1, mutationRate, availableColors))
        next_population.append(mutation(child2, mutationRate, availableColors))

   # next_population[0] = best_individual
    population = next_population

print([p[1] for p in bestPerformers])
print(bestPerformers[-1][0])

[-16, -15, -14, -14, -13, -13, -12, -12, -12, -12, -11, -11, -11, -11, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -9, -9, -9, -9, -9, -9, -9, -9, -9, -9, -8, -8, -8, -8, -8, -8, -8, -8, -8, -8, -8, -8, -8]
[36. 27. 27. 27.  5. 36.  5. 22. 27. 27. 36.  3. 22. 16.  6. 16. 25. 16.
  3. 22.  6. 27. 22.]
